# Classic_Recommendation_Models

In [1]:
# Load the data
import pandas as pd

data_path = 'C:/Users/taek5/Desktop/TG/RecommenderSystems/data/book_crossing/'

books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
users = pd.read_csv(data_path+'BX-Users-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False,encoding="latin-1", index_col = 0)
ratings = pd.read_csv(data_path+'BX-Book-Ratings-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)

C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-

In [2]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_ratings_counts = ratings['Book-Title'].value_counts()
filtered_books = book_ratings_counts[book_ratings_counts >= min_book_ratings].index
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [3]:
# Filter users with at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
ratings = ratings[ratings['User-ID'].isin(filtered_users)]
ratings

,User-ID,Book-Rating,Book-Title
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
5,23768,0,Flesh Tones: A Novel
...,...,...,...
1031125,276688,0,Coyote Waits
1031126,276688,0,Sacred Clowns
1031128,276688,0,Mrs. Pollifax and the Second Thief
1031131,276688,0,Mostly Harmless


In [4]:
# Create the user-item matrix
user_item_matrix = ratings.pivot_table(index = 'User-ID', columns = 'Book-Title', values = 'Book-Rating').fillna(0)

In [13]:
# Split user indices into train and test sets
import numpy as np
from sklearn.model_selection import train_test_split

user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(user_indices, test_size = 0.001, random_state = 42)

In [15]:
def random_recommendation(ratings, n = 10):
    unique_books = ratings['Book-Title'].unique()
    random_books = np.random.choice(unique_books, size = n, replace = False)
    return random_books

random_books = random_recommendation(ratings, n = 10)
print('Random Score-Based Recommendations:')
for i,book in enumerate(random_books, 1):
    print(f"{i}. {book}")

Random Score-Based Recommendations:
1. Das Versprechen. Requiem auf den Kriminalroman.
2. The Delaney Woman
3. In God We Trust: All Others Pay Cash
4. Secrets of the Night
5. Venus Envy
6. The Bishop and the Three Kings: A Blackie Ryan Mystery (Blackie Ryan Novels)
7. The Discworld Mapp
8. The Red Scream
9. Q
10. Hullabaloo in the Guava Orchard


In [16]:
def popularity_recommendation(ratings, n = 10):
    popular_books = ratings.groupby('Book-Title')['Book-Title'].count().sort_values(ascending = False).head(n).index
    return popular_books

popular_books = popularity_recommendation(ratings, n = 10)
print("\nPopularity-Based Recommendations:")
for i, book in enumerate(popular_books, 1):
    print(f"{i}. {book}")


Popularity-Based Recommendations:
1. Wild Animus
2. The Lovely Bones: A Novel
3. The Da Vinci Code
4. The Nanny Diaries: A Novel
5. Bridget Jones's Diary
6. A Painted House
7. The Secret Life of Bees
8. Divine Secrets of the Ya-Ya Sisterhood: A Novel
9. Angels &amp; Demons
10. Life of Pi
